In [1]:
from llmexp.llm.smollm import LLMWrapper
from accelerate import Accelerator
import torch

# checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"
checkpoint = "meta-llama/Llama-3.2-1B-Instruct"
# checkpoint = "HuggingFaceTB/SmolLM-1.7B-Instruct"
saved_mab_model = "checkpoints/mab_model_100.pth"


accelerator = Accelerator()
device = accelerator.device


llm = LLMWrapper(checkpoint, device=device)
tokenizer = llm.tokenizer

In [2]:
# instruction = "Analyze the sentiment of the following sentence. Be brief."
instruction = "Analyze the sentiment of the following sentence and respond with only one word: 'positive,' 'negative,' or 'neutral,' based on the overall tone and meaning of the sentence. Do not provide any additional explanation."
user_input = "I am extremely disappointed with the quality; it broke after just one day."

content = [
            {"role": "system", 
            "content": instruction
            },

            {"role": "sentence", 
            "content": user_input
            }
        ]
template = tokenizer.apply_chat_template(content, tokenize=False, add_generation_prompt=True)
# print(template)

# The generated outputs 
gen_output = llm.generate_from_texts(template)
print(gen_output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond with only one word: 'positive,' 'negative,' or 'neutral,' based on the overall tone and meaning of the sentence. Do not provide any additional explanation.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

I am extremely disappointed with the quality; it broke after just one day.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

negative<|eot_id|>


In [3]:
from llmexp.explainer.mab_model import MABModel
mab_model = MABModel.load_with_base_model(torch.load(saved_mab_model), llm, hidden_size=1024)
mab_model.to(device)
print()

/tmp/1107694.1.gpu/ipykernel_689266/3922162976.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mab_model = MABModel.load_with_base_model(torch.load(saved_mab_model), llm

In [4]:
gen_inputs = tokenizer(gen_output, return_tensors="pt").to(device)
input_ids = gen_inputs['input_ids'][:, :-1]
attention_mask = gen_inputs['attention_mask'][:, :-1]
_, mab_values, _ = mab_model.get_dist_value(input_ids, attention_mask)



We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


In [5]:
print(mab_values)
print(mab_values.shape)

tensor([[ 1.7370e-03,  1.7370e-03,  1.7370e-03,  1.5880e-03, -7.4174e-05,
          2.2015e-03, -8.0579e-04, -1.2981e-03, -2.0936e-03, -7.5222e-03,
         -5.2291e-04, -4.6182e-03, -5.0603e-04,  4.9428e-03,  3.6920e-04,
         -3.9979e-03, -6.3654e-03, -1.0578e-03,  3.4851e-03, -4.1523e-03,
         -2.9535e-03,  3.4223e-03,  5.3811e-03,  2.9356e-03,  4.8862e-03,
         -1.6982e-04, -2.6882e-03, -3.0901e-03, -2.2921e-03, -6.0685e-06,
          3.0490e-04, -7.9522e-04,  6.8277e-04,  1.1475e-03,  1.3192e-03,
         -3.6141e-04,  2.3246e-03, -6.9876e-04,  2.2424e-03,  1.5494e-03,
         -3.1375e-03, -2.6720e-03,  1.3070e-04,  2.8889e-03,  1.0036e-03,
         -1.3501e-03,  1.2568e-03,  2.2978e-03,  1.8504e-03, -1.5304e-03,
         -9.8464e-04, -1.0098e-03,  3.3267e-03,  1.8721e-03,  3.8187e-03,
          1.8039e-03,  1.8381e-03, -3.1492e-03, -1.1453e-03,  1.4353e-03,
         -1.2175e-03,  4.5283e-04, -1.9130e-03,  4.9260e-03,  5.3758e-03,
          2.8541e-04,  4.9535e-03,  7.

In [6]:
print(input_ids)
print(input_ids.shape)

tensor([[128000, 128000, 128000, 128006,   9125, 128007,    271,  38766,   1303,
          33025,   2696,     25,   6790,    220,   2366,     18,    198,  15724,
           2696,     25,    220,   1627,  10263,    220,   2366,     19,    271,
           2127,  56956,    279,  27065,    315,    279,   2768,  11914,    323,
           6013,    449,   1193,    832,   3492,     25,    364,  31587,   2965,
            364,  43324,   2965,    477,    364,  60668,   2965,   3196,    389,
            279,   8244,  16630,    323,   7438,    315,    279,  11914,     13,
           3234,    539,   3493,    904,   5217,  16540,     13, 128009, 128006,
          52989, 128007,    271,     40,   1097,   9193,  25406,    449,    279,
           4367,     26,    433,  14760,   1306,   1120,    832,   1938,     13,
         128009, 128006,  78191, 128007,    271,  43324]], device='cuda:0')
torch.Size([1, 96])


In [7]:
def visualize_tokens_with_values(input_ids, mab_values, tokenizer):
    # Decode tokens one by one to preserve alignment
    tokens = []
    for i in range(input_ids.shape[1]):
        token = tokenizer.decode(input_ids[0, i:i+1])
        tokens.append(token)
    
    # Normalize MAB values to [0,1] for color intensity first
    normalized_values = (mab_values[0] - mab_values[0].min()) / (mab_values[0].max() - mab_values[0].min())
    
    # Pad normalized_values with a zero at the end
    padded_normalized_values = torch.cat([normalized_values, torch.zeros(1, device=mab_values.device)], dim=0)
    # Pad original mab_values with the last actual value
    padded_mab_values = torch.cat([mab_values[0], mab_values[0][-1:]], dim=0)
    
    # Generate HTML with colored text and values
    html_output = "<div style='font-family: monospace; line-height: 2; background-color: white; padding: 10px;'>"
    for token, value, orig_value in zip(tokens, padded_normalized_values, padded_mab_values):
        # Use a gradient from white to green
        intensity = float(value)
        green_color = int(intensity * 200)  # Control the maximum intensity
        html_output += f'<span style="color: black; background-color: rgba(0, {green_color}, 0, 0.3); padding: 0.2em; margin: 0.1em; border-radius: 3px;" title="MAB: {orig_value:.3f}, Norm: {value:.3f}">{token}</span>'
    html_output += "</div>"
    
    # Print the values
    print("Token\tNormalized Value\tOriginal MAB Value")
    print("-" * 50)
    for token, value, orig_value in zip(tokens, padded_normalized_values, padded_mab_values):
        print(f"{token}\t{value:.3f}\t\t{orig_value:.3f}")
    
    from IPython.display import HTML
    return HTML(html_output)

In [8]:
# Usage:
visualization = visualize_tokens_with_values(input_ids, mab_values, tokenizer)
display(visualization)

Token	Normalized Value	Original MAB Value
--------------------------------------------------
<|begin_of_text|>	0.718		0.002
<|begin_of_text|>	0.718		0.002
<|begin_of_text|>	0.718		0.002
<|start_header_id|>	0.706		0.002
system	0.577		-0.000
<|end_header_id|>	0.754		0.002


	0.521		-0.001
Cut	0.482		-0.001
ting	0.421		-0.002
 Knowledge	0.000		-0.008
 Date	0.542		-0.001
:	0.225		-0.005
 December	0.544		-0.001
 	0.966		0.005
202	0.612		0.000
3	0.273		-0.004

	0.090		-0.006
Today	0.501		-0.001
 Date	0.853		0.003
:	0.261		-0.004
 	0.354		-0.003
26	0.848		0.003
 Jul	1.000		0.005
 	0.810		0.003
202	0.962		0.005
4	0.570		-0.000


	0.375		-0.003
An	0.343		-0.003
alyze	0.405		-0.002
 the	0.582		-0.000
 sentiment	0.607		0.000
 of	0.521		-0.001
 the	0.636		0.001
 following	0.672		0.001
 sentence	0.685		0.001
 and	0.555		-0.000
 respond	0.763		0.002
 with	0.529		-0.001
 only	0.757		0.002
 one	0.703		0.002
 word	0.340		-0.003
:	0.376		-0.003
 '	0.593		0.000
positive	0.807		0.003
,'	0.661		0.001
 '	0.